# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 15)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
test_images_dir = "test_images"
os.listdir(test_images_dir)

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
from scipy import stats
from collections import deque

buffer_size = 15
left_slopes = deque(maxlen=buffer_size)
left_intercepts = deque(maxlen=buffer_size)
right_slopes = deque(maxlen=buffer_size)
right_intercepts = deque(maxlen=buffer_size)

def reset_cache():
    for _ in range(buffer_size):
        left_slopes.append(0)
        left_intercepts.append(0)
        right_slopes.append(0)
        right_intercepts.append(0)

# my helper functions

def hacked_grayscale(img):
    copy_image = np.copy(img)
    
    # make darkish pixels gray - i combat shadows with this
    thresholds = (img[:,:,0] < 150) & \
                 (img[:,:,1] < 150) & \
                 (img[:,:,2] < 150)
        
    copy_image[thresholds] = [150, 150, 150]

    # accent bright pixels - i detect yellow lane on sunlight with this
    thresholds = img[:,:,0] > 220
    copy_image[thresholds] = [255, 0, 0]
    thresholds = img[:,:,1] > 220
    copy_image[thresholds] = [0, 255, 0]
    thresholds = img[:,:,2] > 220
    copy_image[thresholds] = [0, 0, 255]
    
    return cv2.cvtColor(copy_image, cv2.COLOR_BGR2GRAY)


def get_awesome_lines(lines, imshape, horizon_offset):
    """""
    
    lines - output of cv2.HoughLinesP
    imshape - originalimage shape
    horizon_offset - y-coordinate where lane lanes approximately end

    Assumptions:
     - left lane segments will have positive slope
     - right lane segments will have negative slopes
     
    What we do:
     - cluster line segments by slope sign
     - fit linear regression to each cluster
     - smooth with historic data from previous frames
    """
    
    horizon_offset -= 15
    
    slope_clusters, point_clusters = sign_slope_clustering(lines, imshape)
     
    slope_left, intercept_left, _, _, _ = stats.linregress([p[0] for p in point_clusters[-1]], 
                                                           [p[1] for p in point_clusters[-1]])
    
    left_slopes.append(slope_left)
    left_intercepts.append(intercept_left)
        
    slope_left = np.mean([x for x in left_slopes if x != 0])
    intercept_left = np.mean([x for x in left_intercepts if x != 0])
            
    left_y_bottom = imshape[0] - 1    
    left_x_bottom = int((left_y_bottom - intercept_left) / slope_left)
    left_y_top = horizon_offset
    left_x_top = int((left_y_top - intercept_left) / slope_left)
    
    slope_right, intercept_right, _, _, _ = stats.linregress([p[0] for p in point_clusters[1]], 
                                                             [p[1] for p in point_clusters[1]])
    
    right_slopes.append(slope_right)
    right_intercepts.append(intercept_right)
    
    slope_right = np.mean([x for x in right_slopes if x != 0])
    intercept_right = np.mean([x for x in right_intercepts if x != 0])
    
    right_y_bottom = imshape[0] - 1    
    right_x_bottom = int((right_y_bottom - intercept_right) / slope_right)
    right_y_top = horizon_offset
    right_x_top = int((right_y_top - intercept_right) / slope_right)
     
    return [[[left_x_bottom, left_y_bottom, left_x_top, left_y_top]],
            [[right_x_bottom, right_y_bottom, right_x_top, right_y_top]]]

    
def sign_slope_clustering(lines, imshape):

    slope_clusters = { 1 : [], 
                      -1 : []}
    
    point_clusters = { 1 : [],
                      -1 : []}
    
    for line in lines:
        x1, y1, x2, y2 = line[0]
        
        if x1 > x2:
            x1, y1, x2, y2 = x2, y2, x1, y1
        
        if (x1 == x2) or (y1 == y2):
            continue    
        
        slope = (y2 - y1) / (x2 - x1)
        slope_sign = np.sign(slope)
        
        # remove anomalous slopes
        if (slope_sign == 1 and (x1 < imshape[1] // 2 or x2 < imshape[1] // 2)) or \
           (slope_sign == -1 and (x1 > imshape[1] // 2 or x2 > imshape[1] // 2)) or \
           not (0.5 < abs(slope) < 0.9):
            continue
                             
        slope_clusters[slope_sign].append(slope)

        point_clusters[slope_sign].append((x1, y1))
        point_clusters[slope_sign].append((x2, y2))
        
    return slope_clusters, point_clusters

In [ ]:
def get_lane_lines(img):
    """
    Returns img with lane lines highlited as red.
    """

    imshape = img.shape

    # gaussian denoising
    
    img_gauss = gaussian_blur(img, 5)
    
    # grayscale
    
    img_gray = hacked_grayscale(img_gauss)
            
    # canny
    
    img_canny = canny(img_gray, 50, 150)
            
    # get region of interest
    
    bottom_x_offset = 55
    middle_y_offset = 75
    middle_x_range = 70
    
    vertices = np.array([[(bottom_x_offset                , imshape[0] - 1), 
                          (imshape[1] // 2 - middle_x_range, imshape[0] // 2 + middle_y_offset), 
                          (imshape[1] // 2 + middle_x_range, imshape[0] // 2 + middle_y_offset), 
                          (imshape[1] - bottom_x_offset   , imshape[0] - 1)]], 
                        dtype=np.int32)
    
    img_region = region_of_interest(img_canny, vertices)
    
    # hough transform
    
    rho = 2             # distance resolution in pixels of the Hough grid
    theta = np.pi / 180 # angular resolution in radians of the Hough grid
    threshold = 15      # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 15   # minimum number of pixels making up a line
    max_line_gap = 30   # maximum gap in pixels between connectable line segments
    
    lines = cv2.HoughLinesP(img_region, 
                            rho, 
                            theta, 
                            threshold, 
                            np.array([]), 
                            minLineLength=min_line_len, 
                            maxLineGap=max_line_gap)
    
    # cluster line segments
    
    lines = get_awesome_lines(lines, imshape, imshape[0] // 2 + middle_y_offset)
    
    # get line image
    
    line_img = np.zeros((*img_region.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines, thickness=10)
                 
    # result!
    
    return weighted_img(line_img, img)

In [ ]:
from PIL import Image
def save_image(img_array, filename):
    im = Image.fromarray(img_array)
    im.save(filename)

In [ ]:
for image_file in os.listdir(test_images_dir):  
    reset_cache()
    name = image_file.partition('.')[0]
    image = mpimg.imread(os.path.join(test_images_dir, image_file))
    image_with_lane_lines = get_lane_lines(image)
    save_image(image_with_lane_lines, os.path.join(test_images_dir, '%s.my.png' % name))

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

    result = get_lane_lines(image)
    
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
reset_cache()
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
reset_cache()
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


# My pipeline:

First I do pretty strong Gaussian blur. Without it I was very fragile to random white lines which occur in the challenge video.

Then I do ugly hacked grayscale conversion. 
First I make darkish pixels gray - this makes black shadows and gray road much more uniform, so Canny stops detecting edges from shadows.
Then I make bright pixels more bright. This helps our yellow line to stand out in the bright sun.
Then I do ordinary grayscale conversion.

Then ordinary steps from tutorials
 - canny
 - region mask
 - hough transform
 
Then I filter line segments with suspicious slopes. This step is really simple but absolutely crucial for robust performance.

Then I cluster line segments by slopes (basically just to separate them into left and right).

After that I get two clusters of points - left and right. We need to average them somehow to get one line in each cluster. First I tried to calculate average slope and average point, but it resulted into wigly, dancing lines on the video. I switched to fitting logistic regression.

After that I still got slight dancing behaviour on videos. An obvious solution was to average lines over past frames.

Assumptions I made:

1) I look for straight lines, not curves.

2) Each frame must contain at least some line segment on the left, and on the right.

3) Camera must be centered. Following from that - if our car will try to change lanes, my algorithm will fail.

4) I'm pretty sure my algorithm is fragile with respect to weather. I have a lot of hardcoded constants, a sure sign      of fragility.

5) Some random obstacle lying on the road will affect my algorithm.

6) If another car will change lanes in front of our car, my algorihtm will fail.

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
reset_cache()
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))